# Exploring Ebay Used Vehicles Data

This project will analyze used vehicles data from eBay Kleinanzeigen (eBay Germany). The dataset was originally scraped (March, 5, 2016 through April, 7, 2016) and uploaded to  <a href="https://www.kaggle.com/orgesleka/used-cars-database/data">Kaggle</a>. The data contains several variables describing features about the vehicle in the ad (e.g., model, gas type, kilometers driven, etc), and variables about the website (e.g., type of seller, type of listing, number of pictures in the ad, etc). Here is the data dictionary associated with the data:

 - __dateCrawled:__ When this ad was first crawled. All field-values are taken from this date.
 - __name:__  Name of the car.
 - __seller:__ Whether the seller is private or a dealer.
 - __offerType :__ The type of listing
 - __price:__ The price on the ad to sell the car.
 - __abtest:__ Whether the listing is included in an A/B test.
 - __vehicleType:__ The vehicle Type.
 - __yearOfRegistration:__ The year in which the car was first registered.
 - __gearbox:__ The transmission type.
 - __powerPS:__ The power of the car in PS.
 - __model:__ The car model name.
 - __kilometer:__ How many kilometers the car has driven.
 - __monthOfRegistration:__ The month in which the car was first registered.
 - __fuelType:__ What type of fuel the car uses.
 - __brand:__ The brand of the car.
 - __notRepairedDamage:__ If the car has a damage which is not yet repaired.
 - __dateCreated:__ The date on which the eBay listing was created.
 - __nrOfPictures:__ The number of pictures in the ad.
 - __postalCode:__ The postal code for the location of the vehicle.
 - __lastSeenOnline:__ When the crawler saw this ad last online.


# Objective
The aim of this project is to clean the data and analyze the included used car listings.

In [66]:
import pandas as pd
import numpy as np

In [67]:
autos = pd.read_csv("autos.csv", encoding="Latin-1")

In [68]:
# There shouldn't be duplicates given each row represents one unique ad. 
autos[autos.duplicated()]

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen


In [69]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

The dataset has __50.000__ entires and is fromed by __20__ columns (Types: int64 and object)

In [70]:
autos.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [71]:
autos["notRepairedDamage"].value_counts()

nein    35232
ja       4939
Name: notRepairedDamage, dtype: int64

- Columns like dateCrawled, dateCreated and lastSeen are __object__ type, but should be __datetime__ type. To work easier with them.
- Columns like vechicleType, gearbox, model or notRepairedDamage have missing values for some entries.
- Columns like price or odometer have special characters ($, km)
- Columns like notRepairedDamage contain German words (nein - no, ja - yes)

## Cleaning the Data

In [72]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [73]:
columns_names = ['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'nr_of_pictures', 'postal_code',
       'last_seen']

In [74]:
autos.columns = columns_names 
autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


To make the dataset more understandable some some columns needed a name change.

And all the columns were changed from camelcase to snakecase following the Python standards

In [9]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-08 10:40:35,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


In [10]:
autos.describe()

,registration_year,power_ps,registration_month,nr_of_pictures,postal_code
count,50000.000000,50000.000000,50000.000000,50000.0,50000.000000
mean,2005.073280,116.355920,5.723360,0.0,50813.627300
std,105.712813,209.216627,3.711984,0.0,25779.747957
min,1000.000000,0.000000,0.000000,0.0,1067.000000
25%,1999.000000,70.000000,3.000000,0.0,30451.000000
50%,2003.000000,105.000000,6.000000,0.0,49577.000000
75%,2008.000000,150.000000,9.000000,0.0,71540.000000
max,9999.000000,17700.000000,12.000000,0.0,99998.000000


In [75]:
autos["nr_of_pictures"].value_counts()

0    50000
Name: nr_of_pictures, dtype: int64

In [76]:
autos.drop(["nr_of_pictures"], axis = 1, inplace = True)

The column __nr_of_pictures__ is dropped because contais only one value __0__, which doesn't give any information.

In [77]:
autos[["price", "odometer"]]

,price,odometer
0,"$5,000","150,000km"
1,"$8,500","150,000km"
2,"$8,990","70,000km"
3,"$4,350","70,000km"
4,"$1,350","150,000km"
...,...,...
49995,"$24,900","100,000km"
49996,"$1,980","150,000km"
49997,"$13,200","5,000km"
49998,"$22,900","40,000km"


The columns __seller__ and __offer_type__ have the same values.
The columns __price__ and __odometer__ can be converted to numeric types.

In [78]:
autos["price"].head(5)

0    $5,000
1    $8,500
2    $8,990
3    $4,350
4    $1,350
Name: price, dtype: object

In [79]:
autos["price"] = autos["price"].str.replace("$", "")
autos["price"] = autos["price"].str.replace(",", "").astype(int)
autos["price"].head(5)

0    5000
1    8500
2    8990
3    4350
4    1350
Name: price, dtype: int64

In [80]:
autos["odometer"].head(5)

0    150,000km
1    150,000km
2     70,000km
3     70,000km
4    150,000km
Name: odometer, dtype: object

In [81]:
autos["odometer"] = autos["odometer"].str.replace("km", "")
autos["odometer"] = autos["odometer"].str.replace(",", "")
autos["odometer"].astype(int)
autos = autos.rename(columns={'odometer': 'odometer_km'})
autos["odometer_km"].head(5)

0    150000
1    150000
2     70000
3     70000
4    150000
Name: odometer_km, dtype: object

In [82]:
autos["price"].unique().shape

(2357,)

In [83]:
autos["odometer_km"].unique().shape

(13,)

In [84]:
autos["price"].describe()

count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64

In [85]:
autos["odometer_km"].describe()

count      50000
unique        13
top       150000
freq       32424
Name: odometer_km, dtype: object

In [86]:
autos["price"].value_counts().sort_index(ascending=False).head()

99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
Name: price, dtype: int64

In [87]:
autos["odometer_km"].value_counts().sort_index(ascending=False)

90000      1757
80000      1436
70000      1230
60000      1164
50000      1027
5000        967
40000       819
30000       789
20000       784
150000    32424
125000     5170
100000     2169
10000       264
Name: odometer_km, dtype: int64

Knowing that eBay is an auction website, it is possible to have these low values in the price column. Several of these entries appear to be illegitimate entries (e.g., 1234566, 12345678, 1111111, 99999999). In addition, it is unlikely that these prices represent the value of the vehicles given that this is a used vehicle classified site. Therefore, anything below __1.00__ and above __350.000__ will be removed as these could have been errors or illegitimate entries.

For the odometer column, the distribution of kilometers is skewed towards the high end. Perhaps this suggests that many of these vehicles are classics, or that people want to sell their vehicles for used parts.

In [88]:
autos[autos["price"] == 0]

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
27,2016-03-27 18:45:01,Hat_einer_Ahnung_mit_Ford_Galaxy_HILFE,privat,Angebot,0,control,NaN,2005,NaN,0,NaN,150000,0,NaN,ford,NaN,2016-03-27 00:00:00,66701,2016-03-27 18:45:01
71,2016-03-28 19:39:35,Suche_Opel_Astra_F__Corsa_oder_Kadett_E_mit_Re...,privat,Angebot,0,control,NaN,1990,manuell,0,NaN,5000,0,benzin,opel,NaN,2016-03-28 00:00:00,4552,2016-04-07 01:45:48
80,2016-03-09 15:57:57,Nissan_Primera_Hatchback_1_6_16v_73_Kw___99Ps_...,privat,Angebot,0,control,coupe,1999,manuell,99,primera,150000,3,benzin,nissan,ja,2016-03-09 00:00:00,66903,2016-03-09 16:43:50
87,2016-03-29 23:37:22,Bmw_520_e39_zum_ausschlachten,privat,Angebot,0,control,NaN,2000,NaN,0,5er,150000,0,NaN,bmw,NaN,2016-03-29 00:00:00,82256,2016-04-06 21:18:15
99,2016-04-05 09:48:54,Peugeot_207_CC___Cabrio_Bj_2011,privat,Angebot,0,control,cabrio,2011,manuell,0,2_reihe,60000,7,diesel,peugeot,nein,2016-04-05 00:00:00,99735,2016-04-07 12:17:34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49884,2016-03-11 13:55:30,Audi_a6_2.5l__Schnaeppchen_nur_heute,privat,Angebot,0,test,kombi,1999,manuell,150,a6,150000,11,diesel,audi,NaN,2016-03-11 00:00:00,27711,2016-03-12 03:17:08
49943,2016-03-16 20:46:08,Opel_astra,privat,Angebot,0,control,NaN,2016,manuell,101,astra,150000,8,benzin,opel,NaN,2016-03-16 00:00:00,89134,2016-03-17 19:44:20
49960,2016-03-25 22:51:55,Ford_KA_zu_verschenken_***Reserviert***,privat,Angebot,0,control,kleinwagen,1999,manuell,60,ka,150000,6,benzin,ford,NaN,2016-03-25 00:00:00,34355,2016-03-25 22:51:55
49974,2016-03-20 10:52:31,Golf_1_Cabrio_Tuev_Neu_viele_Extras_alles_eing...,privat,Angebot,0,control,cabrio,1983,manuell,70,golf,150000,2,benzin,volkswagen,nein,2016-03-20 00:00:00,8209,2016-03-27 19:48:16


In [89]:
autos = autos[(autos.loc[:, "price"] > 0) & (autos.loc[:, "price"] <= 350000)]
autos["price"].value_counts().head(5)

500     781
1500    734
2500    643
1200    639
1000    639
Name: price, dtype: int64

In [90]:
autos["odometer_km"].describe()

count      48565
unique        13
top       150000
freq       31414
Name: odometer_km, dtype: object

In [91]:
autos["price"].describe()

count     48565.000000
mean       5888.935591
std        9059.854754
min           1.000000
25%        1200.000000
50%        3000.000000
75%        7490.000000
max      350000.000000
Name: price, dtype: float64

In [92]:
autos["date_crawled"].str[:10].value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.025327
2016-03-06    0.014043
2016-03-07    0.036014
2016-03-08    0.033296
2016-03-09    0.033090
2016-03-10    0.032184
2016-03-11    0.032575
2016-03-12    0.036920
2016-03-13    0.015670
2016-03-14    0.036549
2016-03-15    0.034284
2016-03-16    0.029610
2016-03-17    0.031628
2016-03-18    0.012911
2016-03-19    0.034778
2016-03-20    0.037887
2016-03-21    0.037373
2016-03-22    0.032987
2016-03-23    0.032225
2016-03-24    0.029342
2016-03-25    0.031607
2016-03-26    0.032204
2016-03-27    0.031092
2016-03-28    0.034860
2016-03-29    0.034099
2016-03-30    0.033687
2016-03-31    0.031834
2016-04-01    0.033687
2016-04-02    0.035478
2016-04-03    0.038608
2016-04-04    0.036487
2016-04-05    0.013096
2016-04-06    0.003171
2016-04-07    0.001400
Name: date_crawled, dtype: float64

The __date_crawled__ column range goes from 2016-04-05 to 2016-04-07

In [93]:
autos["ad_created"].str[:10].value_counts(normalize=True, dropna=False).sort_index()

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
                ...   
2016-04-03    0.038855
2016-04-04    0.036858
2016-04-05    0.011819
2016-04-06    0.003253
2016-04-07    0.001256
Name: ad_created, Length: 76, dtype: float64

The __ad_created__ range goes from 2015-06-11 to 2016-04-07

In [94]:
autos["last_seen"].str[:10].value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.001071
2016-03-06    0.004324
2016-03-07    0.005395
2016-03-08    0.007413
2016-03-09    0.009595
2016-03-10    0.010666
2016-03-11    0.012375
2016-03-12    0.023783
2016-03-13    0.008895
2016-03-14    0.012602
2016-03-15    0.015876
2016-03-16    0.016452
2016-03-17    0.028086
2016-03-18    0.007351
2016-03-19    0.015834
2016-03-20    0.020653
2016-03-21    0.020632
2016-03-22    0.021373
2016-03-23    0.018532
2016-03-24    0.019767
2016-03-25    0.019211
2016-03-26    0.016802
2016-03-27    0.015649
2016-03-28    0.020859
2016-03-29    0.022341
2016-03-30    0.024771
2016-03-31    0.023783
2016-04-01    0.022794
2016-04-02    0.024915
2016-04-03    0.025203
2016-04-04    0.024483
2016-04-05    0.124761
2016-04-06    0.221806
2016-04-07    0.131947
Name: last_seen, dtype: float64

The __last_seen__ range goes from 2016-03-05 to 2016-04-07

In [95]:
autos["registration_year"].describe()

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

In [96]:
autos["registration_year"].head(5)

0    2004
1    1997
2    2009
3    2007
4    2003
Name: registration_year, dtype: int64

The highest registration year acceptable: 2016 (current year of creation of the dataset)
The lowest registration year acceptable: 1990  (when first cars where made)

In [97]:
autos[(autos["registration_year"] <= 2016) & (autos["registration_year"] >= 1900)]

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer_km,registration_month,fuel_type,brand,unrepaired_damage,ad_created,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,5000,control,bus,2004,manuell,158,andere,150000,3,lpg,peugeot,nein,2016-03-26 00:00:00,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,8500,control,limousine,1997,automatik,286,7er,150000,6,benzin,bmw,nein,2016-04-04 00:00:00,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,8990,test,limousine,2009,manuell,102,golf,70000,7,benzin,volkswagen,nein,2016-03-26 00:00:00,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,4350,control,kleinwagen,2007,automatik,71,fortwo,70000,6,benzin,smart,nein,2016-03-12 00:00:00,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,1350,test,kombi,2003,manuell,0,focus,150000,7,benzin,ford,nein,2016-04-01 00:00:00,39218,2016-04-01 14:38:50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,2016-03-27 14:38:19,Audi_Q5_3.0_TDI_qu._S_tr.__Navi__Panorama__Xenon,privat,Angebot,24900,control,limousine,2011,automatik,239,q5,100000,1,diesel,audi,nein,2016-03-27 00:00:00,82131,2016-04-01 13:47:40
49996,2016-03-28 10:50:25,Opel_Astra_F_Cabrio_Bertone_Edition___TÜV_neu+...,privat,Angebot,1980,control,cabrio,1996,manuell,75,astra,150000,5,benzin,opel,nein,2016-03-28 00:00:00,44807,2016-04-02 14:18:02
49997,2016-04-02 14:44:48,Fiat_500_C_1.2_Dualogic_Lounge,privat,Angebot,13200,test,cabrio,2014,automatik,69,500,5000,11,benzin,fiat,nein,2016-04-02 00:00:00,73430,2016-04-04 11:47:27
49998,2016-03-08 19:25:42,Audi_A3_2.0_TDI_Sportback_Ambition,privat,Angebot,22900,control,kombi,2013,manuell,150,a3,40000,11,diesel,audi,nein,2016-03-08 00:00:00,35683,2016-04-05 16:45:07


In [98]:
autos.drop(autos[(autos["registration_year"] < 1940) | (autos["registration_year"] > 2016)].index, inplace=True)

In [99]:
autos["registration_year"].describe()

count    46665.000000
mean      2002.936826
std          7.043626
min       1941.000000
25%       1999.000000
50%       2003.000000
75%       2008.000000
max       2016.000000
Name: registration_year, dtype: float64

In [100]:
autos["registration_year"].value_counts(normalize=True)

2000    0.067631
2005    0.062917
1999    0.062081
2004    0.057923
2003    0.057838
          ...   
1941    0.000043
1948    0.000021
1943    0.000021
1953    0.000021
1952    0.000021
Name: registration_year, Length: 70, dtype: float64

In [101]:
autos.isnull().sum()

date_crawled             0
name                     0
seller                   0
offer_type               0
price                    0
abtest                   0
vehicle_type          2700
registration_year        0
gearbox               2103
power_ps                 0
model                 2185
odometer_km              0
registration_month       0
fuel_type             3312
brand                    0
unrepaired_damage     8299
ad_created               0
postal_code              0
last_seen                0
dtype: int64

The columns entries: __vehicle_type (2700)__, __gearbox (2103)__, __model (2185)__, __fuel_type (3312)__, __unrepaired_damage (8299)__ with null values.

In [102]:
autos["brand"].nunique()

40

In [103]:
autos["brand"].unique()

array(['peugeot', 'bmw', 'volkswagen', 'smart', 'ford', 'chrysler',
       'seat', 'renault', 'mercedes_benz', 'audi', 'sonstige_autos',
       'opel', 'mazda', 'porsche', 'mini', 'toyota', 'dacia', 'nissan',
       'jeep', 'saab', 'volvo', 'mitsubishi', 'jaguar', 'fiat', 'skoda',
       'subaru', 'kia', 'citroen', 'chevrolet', 'hyundai', 'honda',
       'daewoo', 'suzuki', 'trabant', 'land_rover', 'alfa_romeo', 'lada',
       'rover', 'daihatsu', 'lancia'], dtype=object)

In [104]:
autos["vehicle_type"].nunique()

8

In [105]:
autos["vehicle_type"].unique()

array(['bus', 'limousine', 'kleinwagen', 'kombi', nan, 'coupe', 'suv',
       'cabrio', 'andere'], dtype=object)

There are __8__ vehicle types (nan value included) and __40__ vehicle brands. 

In [106]:
autos["name"].nunique()

35801

In [107]:
autos["name"].unique()

array(['Peugeot_807_160_NAVTECH_ON_BOARD',
       'BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik',
       'Volkswagen_Golf_1.6_United', ...,
       'Audi_Q5_3.0_TDI_qu._S_tr.__Navi__Panorama__Xenon',
       'Opel_Astra_F_Cabrio_Bertone_Edition___TÜV_neu+Reifen_neu_!!',
       'Fiat_500_C_1.2_Dualogic_Lounge'], dtype=object)

In [116]:
words_dictionary = {"privat": "private", "gewerblich": "commercial", "Angebot": "bid", "Gesuch": "application", "kleinwagen": 
            "super mini", "kombi":"station wagon", "cabrio": "convertible", "limousine": "sedan", "andere": "other", "manuell": "manual", "automatik": 
            "automatic", "benzin":"gas", "elektro": "electric", "sonstige_auto": "other", "sonstige_autos": "other", "nein": "no", "ja": "yes"}

def translate(value):
    if value in words_dictionary:
        return words_dictionary[value]
    else:
        return value
    
columns_to_change = ["seller", "offer_type", "vehicle_type", "gearbox", "model", "fuel_type", "brand", "unrepaired_damage"]
autos[columns_to_change] = autos[columns_to_change].applymap(translate)

In [117]:
autos["name"].head(30)

0                      Peugeot_807_160_NAVTECH_ON_BOARD
1            BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik
2                            Volkswagen_Golf_1.6_United
3     Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...
4     Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...
5     Chrysler_Grand_Voyager_2.8_CRD_Aut.Limited_Sto...
6     VW_Golf_III_GT_Special_Electronic_Green_Metall...
7                                  Golf_IV_1.9_TDI_90PS
8                                            Seat_Arosa
9             Renault_Megane_Scenic_1.6e_RT_Klimaanlage
11                           Mercedes_A140_Motorschaden
12    Smart_smart_fortwo_coupe_softouch_pure_MHD_Pan...
13                                   Audi_A3_1.6_tuning
14    Renault_Clio_3__Dynamique_1.2__16_V;_viele_Ver...
15          Corvette_C3_Coupe_T_Top_Crossfire_Injection
16                                  Opel_Vectra_B_Kombi
17                            Volkswagen_Scirocco_2_G60
18                 Verkaufen_mein_bmw_e36_320_i_

After translating German names to English, in the column __names__ appears to contain __vehicle_type__ values.

In [118]:
import warnings
warnings.filterwarnings("ignore", 'This pattern has match groups')
# Used str.contains as a make-shift boolean index, hence disabling the warnings. 

In [119]:
# Replaced null values in vehicle type based word matches in name 
autos.loc[autos["name"].str.contains(r"(.*kombi)", case = False), "vehicle_type"] = "station wagon"
autos.loc[autos["name"].str.contains(r"(.*cabrio)", case = False), "vehicle_type"] = "convertible"
autos.loc[autos["name"].str.contains(r"(.*klein)", case = False), "vehicle_type"] = "small car"
autos.loc[autos["name"].str.contains(r"(.*limo[^n])", case = False), "vehicle_type"] = "sedan"
autos.loc[autos["name"].str.contains(r"(.*coupe)", case = False), "vehicle_type"] = "coupe"
autos.loc[autos["name"].str.contains(r"(.*bus)", case = False), "vehicle_type"] = "bus"
autos.loc[autos["name"].str.contains(r"(.*caravan)", case = False), "vehicle_type"] = "van"
# Multiple instances where there could be multiple vehicle type keywords in a string. Thus, "smart car" was replaced to "small car" last
# to reverse some of these cleaning induced errors.
autos.loc[autos["name"].str.contains(r"(.*smart)", case = False), "vehicle_type"] = "small car"

In [120]:
# Reactivated filtering warnings
import warnings
warnings.filterwarnings("default", 'This pattern has match groups')

In [121]:
autos.isnull().sum()

date_crawled             0
name                     0
seller                   0
offer_type               0
price                    0
abtest                   0
vehicle_type          2493
registration_year        0
gearbox               2103
power_ps                 0
model                 2185
odometer_km              0
registration_month       0
fuel_type             3312
brand                    0
unrepaired_damage     8299
ad_created               0
postal_code              0
last_seen                0
dtype: int64

The __null__ values from the __vehicle_type__ column where reduced from __2700__ to __2493__.

In [122]:
autos.columns

Index(['date_crawled', 'name', 'seller', 'offer_type', 'price', 'abtest',
       'vehicle_type', 'registration_year', 'gearbox', 'power_ps', 'model',
       'odometer_km', 'registration_month', 'fuel_type', 'brand',
       'unrepaired_damage', 'ad_created', 'postal_code', 'last_seen'],
      dtype='object')

In [52]:
autos["brand"].value_counts().head(20).index

Index(['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford', 'renault',
       'peugeot', 'fiat', 'seat', 'skoda', 'nissan', 'mazda', 'smart',
       'citroen', 'toyota', 'hyundai', 'sonstige_autos', 'volvo', 'mini'],
      dtype='object')

In [53]:
# After seeing the value counts of the brand column, 
# I decided to agrregate the first 20 brands of the value counts index
# Which are the ones which more percentage of the total value.

In [54]:
# Brands list
brands = [ 'volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi', 'ford', 'renault',
       'peugeot', 'fiat', 'seat', 'skoda', 'nissan', 'mazda', 'smart',
       'citroen', 'toyota', 'hyundai', 'sonstige_autos', 'volvo', 'mini']

In [55]:
# Creates the mean price of a concrete brand
autos[autos["brand"] == 'volkswagen']["price"].mean()

6729.81956411556

In [56]:
#Brands dict
brands_dict = {}
for brand in brands:
    brands_dict[brand] = autos[autos["brand"] == brand]["price"].mean()

brands_dict

{'audi': 9336.687453600594,
 'bmw': 8570.910064239828,
 'citroen': 45486.68549618321,
 'fiat': 2813.748538011696,
 'ford': 7447.770999386879,
 'hyundai': 5365.254273504273,
 'mazda': 4112.596614950635,
 'mercedes_benz': 30834.80146568954,
 'mini': 10613.459657701711,
 'nissan': 4743.40252454418,
 'opel': 5434.952788844622,
 'peugeot': 3094.0172290021537,
 'renault': 2475.7172727272728,
 'seat': 4397.230949589683,
 'skoda': 6368.0,
 'smart': 3580.2239031770046,
 'sonstige_autos': 45644.97802197802,
 'toyota': 5167.091062394604,
 'volkswagen': 6729.81956411556,
 'volvo': 4946.501170960188}

In [57]:
autos["brand"].value_counts().head(20)

volkswagen        9865
bmw               5137
opel              5020
mercedes_benz     4503
audi              4041
ford              3262
renault           2200
peugeot           1393
fiat              1197
seat               853
skoda              766
nissan             713
mazda              709
smart              661
citroen            655
toyota             593
hyundai            468
sonstige_autos     455
volvo              427
mini               409
Name: brand, dtype: int64

In [58]:
# As we can see in the previous list the car whith higher mean price is
# is Audi, which is not the one with most appearances, that is volkswagen
# and has a lower price. That means that volksvagen cobines quality & price.

In [72]:
# Only top 6 brands
brands = ["audi", "bmw", "ford", "mercedes_benz", "opel", "volkswagen"]

In [73]:
price_dict = {}
for brand in brands:
    price_dict[brand] = autos[autos["brand"] == brand]["price"].mean()

price_dict

{'audi': 9336.687453600594,
 'bmw': 8570.910064239828,
 'ford': 7447.770999386879,
 'mercedes_benz': 30834.80146568954,
 'opel': 5434.952788844622,
 'volkswagen': 6729.81956411556}

In [74]:
mileage_dict = {}
for brand in brands:
    mileage_dict[brand] = autos[autos["brand"] == brand]["odometer_km"].mean()

mileage_dict

{'audi': 129157.38678544914,
 'bmw': 132600.73973136072,
 'ford': 124351.62477007971,
 'mercedes_benz': 130820.56406839885,
 'opel': 129340.63745019921,
 'volkswagen': 128713.6340598074}

In [75]:
price_series = pd.Series(price_dict)
price_series

audi              9336.687454
bmw               8570.910064
ford              7447.770999
mercedes_benz    30834.801466
opel              5434.952789
volkswagen        6729.819564
dtype: float64

In [76]:
mileage_series = pd.Series(mileage_dict)
mileage_series

audi             129157.386785
bmw              132600.739731
ford             124351.624770
mercedes_benz    130820.564068
opel             129340.637450
volkswagen       128713.634060
dtype: float64

In [77]:
df = pd.DataFrame(price_series, columns=['mean_price'])
df

,mean_price
audi,9336.687454
bmw,8570.910064
ford,7447.770999
mercedes_benz,30834.801466
opel,5434.952789
volkswagen,6729.819564


In [78]:
df["mileage"] = mileage_series
df

,mean_price,mileage
audi,9336.687454,129157.386785
bmw,8570.910064,132600.739731
ford,7447.770999,124351.624770
mercedes_benz,30834.801466,130820.564068
opel,5434.952789,129340.637450
volkswagen,6729.819564,128713.634060
